In [18]:
import sounddevice as sd
import random
import numpy as np
import librosa

from threading import Thread
from pydub import AudioSegment
from scipy.io import wavfile
from midi2audio import FluidSynth
from midiutil import MIDIFile
from src.util.features import get_feature_vector
from scipy.spatial.distance import cosine, minkowski
from io import BytesIO

In [174]:
FONT = "./src/soundfonts/piano_eletro.sf2"
MIDI_FILE = "./src/input/temp"
WAV_FILE = "./src/output/temp"

In [42]:
def loop_play(file=None, sr=44100):
    t = Thread(target=play)
    t.start()

In [20]:
def play(file=None, sr=44100):
    if file == None: file=WAV_FILE+".wav"
    song, fs = librosa.load(file,sr=sr)
    sd.play(song, fs, loop=True)
    sd.wait()

In [21]:
def rand(low=0, high=2):
    return random.random()*(high - low + 1) + low

In [22]:
def save_harmony(midi_file, out):
    with open(MIDI_FILE + out + ".midi", "wb") as output_file:
        midi_file.writeFile(output_file)
        output_file.close()
    
    song = BytesIO()
    wav_song = BytesIO()
    
    midi_file.writeFile(song)

    FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + out + ".midi", WAV_FILE + out + ".wav")

## Parameters to encode
 - [ ] Tempo
 - [ ] Nº of Notes
 - [x] Notes
 - [x] Notes duration
 - [x] Probability of operations 

## Methods For the Generation of the chromossome

In [150]:
MAX_NOTE = 81
MIN_NOTE = 60

In [151]:
list(range(MIN_NOTE-1, MAX_NOTE, 12))

[59, 71]

In [164]:
def get_random_pitch():
    pitch_base = list(range(MIN_NOTE-1, MAX_NOTE, 12))
    n = random.choice(range(len(pitch_base)))
    return pitch_base[n]

In [165]:
def generate_random_note():
    pitch = get_random_pitch()
    notes = [1, 3, 5, 6, 8, 10, 12] #notes in C-major scale
    note = random.choice(notes) + pitch
    return note

In [166]:
def generate_random_duration():
    #base = generate_random_note()
    #chord = [base, base+4, base+7]
    duration = random.choice(range(1, 8+1))
    return duration#, chord

In [206]:
def generate_random_genome():
    N = 5
    probs = [
        np.array([0.4,   #P(mutation)
                  0.2,   #P(crossover) 
                  0.2,   #P(inverse)
                  0.2]), #P(duplication) 
        np.full(shape=11, fill_value=1/11)  # P(m_i|Mutation)
            ]
    genome = [*probs]
    for _ in range(N):
        duration = generate_random_duration()
        melody_note = generate_random_note()
        genome.append( (duration, melody_note) )
    
    return genome

In [208]:
def split_genome(gen):
    return gen[0], gen[1], gen[2:]

In [209]:
def get_phenotype(genome):
    midi_file = MIDIFile(2)
    melody=0
    
    tempo = 120
    midi_file.addTempo(melody, 0, tempo)
    midi_file.addTempo(1,0,tempo)
    
    cum_time = 0
    
    prob_op, prob_mut, notes = split_genome(genome)
    for el in notes:
        d, note= el
        midi_file.addNote(melody, 0, note, cum_time, d/2, 100)
        
        cum_time+=d/2
    save_harmony(midi_file, "")
    return midi_file

In [210]:
gen = generate_random_genome()
gen

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [145]:
gen_2=passing_tone(gen)
gen_2

[(0.4, 0.2, 0.2, 0.2),
 (2, [29, 33, 36], 74),
 (1.0, [29, 33, 36], 76),
 (1.0, [29, 33, 36], 77),
 (1.0, [29, 33, 36], 79),
 (1.0, [29, 33, 36], 81),
 (1.0, [29, 33, 36], 83),
 (1.0, [29, 33, 36], 84),
 (1.0, [29, 33, 36], 86),
 (6, [28, 32, 35], 88),
 (5, [41, 45, 48], 52),
 (2, [62, 66, 69], 69),
 (6, [55, 59, 62], 41)]

In [175]:
mf = get_phenotype(gen)

## Selection Method

In [260]:
5*0.1

0.5

In [299]:
fitness = lambda g: rand(0,5)

In [261]:
def truncate(population_fitness, N):
    to_remove = max(N//10, 1)
    population_fitness = population_fitness[:-to_remove]
    
    return population_fitness

In [348]:
def elitism(population_fitness: list, N: int) -> list:
    to_keep = max(N//10, 1)
    new_population = population_fitness[:to_keep]
    
    return list(map(lambda x: x[0], new_population))

In [384]:
def tournament(population_fitness, k):    
    indexes = [i for i in range(len(population_fitness))]
    
    selected_indexes = list(np.random.choice(
        indexes, 
        size=k, 
        replace=False
    ))
    population_fitness_subset = [ population_fitness[i] for i in selected_indexes]
    population_fitness_subset.sort(key = lambda x: x[1])    
    genome_selected = population_fitness_subset[0]
    return genome_selected[0]

In [ ]:
population = [ generate_random_genome() for _ in range(20)]

In [397]:
def selection(population):
    N = len(population)
    
    population_fitness = list(map(lambda gen: (gen, fitness(gen)), population))
    population_fitness.sort(key = lambda x: x[1])
    population_fitness = truncate(population_fitness, N)
    
    new_population = elitism(population_fitness, N)
    
    k_left = N - len(new_population)
    while len(new_population) < N:
        genome = tournament(population_fitness, 2)
        
        probs_op, probs_mut, notes = split_genome(genome)
        operator = select_operator(genome, population_fitness, k_left)
        new_genomes = operator(genome, population_fitness, N) 
        
        for genome in new_genomes:
            new_population.append(genome)
            k_left-=1
    return new_population

In [398]:
selection(population)

[[array([0.4, 0.2, 0.2, 0.2]),
  array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909]),
  (3, 60),
  (5, 64),
  (1, 69),
  (4, 64),
  (6, 62)],
 [array([0.4, 0.2, 0.2, 0.2]),
  array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909]),
  (1, 71),
  (1, 67),
  (7, 69),
  (1, 62),
  (5, 62)],
 [array([0.4, 0.2, 0.2, 0.2]),
  array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909]),
  (7, 83),
  (2, 79),
  (3, 72),
  (8, 62),
  (4, 65)],
 [array([0.4, 0.2, 0.2, 0.2]),
  array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
         0.09090909]),
  (5, 81),
  (2, 71),
  (3, 79),
  (3, 64),
  (1, 69),
  (4, 64)

In [400]:
get_phenotype(population[-11])

In [379]:
gen = generate_random_genome()

## Operators

In [395]:
def select_operator(genome, population_fitness, k_left):
    prob_op, prob_mut, notes = split_genome(genome)
    if k_left>=2:
        operator_index = np.random.choice( range(4), size=1, p=prob_op )[0]
    else:
        available = [0, 2, 3]
        temp_prob = prob_op[available]
        temp_prob /= temp_prob.sum()
        operator_index = np.random.choice( available, size=1, p=temp_prob)[0]
        
    operators = [ mutation, crossover, duplication, inversion ]
    return operators[ operator_index ]

In [255]:
operation(gen)

[array([0.39045189, 0.14255212, 0.18181104, 0.28518495]),
 array([0.03640833, 0.00605717, 0.06913054, 0.2475582 , 0.07673454,
        0.09387541, 0.08775609, 0.08151578, 0.13514298, 0.08694019,
        0.07888077]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

## Methods Regarding Mutation

In [211]:
def repeat(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    notes.insert(n, notes[n])
    genome = [prob_op, prob_mut, *notes]
    return genome

In [212]:
repeat(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71),
 (4, 71)]

In [217]:
def split(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    duration, note = notes[n]
    new_duration = duration/2
    
    new_note = (new_duration, note)
    notes[n] = new_note
    notes.insert(n, new_note)
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [218]:
split(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (2.0, 77),
 (2.0, 77),
 (4, 71)]

In [219]:
def arpeggiate(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    duration, note = notes[n]
    pitch_incr = random.choice([4,7])
    
    new_pitch = min(MAX_NOTE, note+pitch_incr)
    new_note = (duration, new_pitch)
    
    notes.insert(n+1, new_note)
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [220]:
arpeggiate(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (3, 68),
 (4, 77),
 (4, 71)]

In [221]:
def leap(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    duration, note = notes[n]
    while (new:=generate_random_note())==note: continue
    
    new_note = (duration, new)
    notes[n] = new_note
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [222]:
leap(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 69),
 (4, 71)]

In [223]:
def get_pairs(notes):
    is_consecutive = []
    for i in range(len(notes)-1):
        d_a, note_a = notes[i]
        d_b, note_b = notes[i+1]
        
        if note_a==note_b:
            is_consecutive.append(i+1)
    return is_consecutive

In [224]:
def diatonic_upper_step_size(note):
    if note%12 in (0, 2, 5, 7, 9):
        return 2
    return 1

In [225]:
def diatonic_lower_step_size(note):
    if note%12 in (0, 5):
        return 1
    return 2

In [226]:
def upper_neighbor(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    
    is_consecutive = get_pairs(notes)
    if len(is_consecutive)==0:
        return genome
    
    n = random.choice(is_consecutive)
    d, note = notes[n]
    
    step = diatonic_upper_step_size(note)
    note = min(MAX_NOTE, note+step)
    
    notes[n] = (d, note)
    genome = [prob_op, prob_mut, *notes]
    return genome   

In [228]:
upper_neighbor(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [227]:
def lower_neighbor(genome):
    prob_op, prob_mut, notes = split_genome(genome)

    is_consecutive = get_pairs(notes)
    if len(is_consecutive)==0:
        return genome
    
    n = random.choice(is_consecutive)
    d, note = notes[n]
    
    step = diatonic_lower_step_size(note)
    note = max(MIN_NOTE, note - step)
    
    notes[n] = (d, note)
    genome = [prob_op, prob_mut, *notes]
    return genome

In [229]:
lower_neighbor(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [230]:
def anticipation(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    duration, note = notes[n]
    
    notes[n] = (duration * 0.25, note)
    notes.insert(n+1, (duration * 0.75, note))
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [231]:
anticipation(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (0.75, 64),
 (2.25, 64),
 (4, 77),
 (4, 71)]

In [232]:
def delay(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    duration, note = notes[n]
    
    notes[n] = (duration * 0.75, note)
    notes.insert(n+1, (duration * 0.25, note))
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [233]:
delay(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (3.75, 81),
 (1.25, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [234]:
def passing_tone(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice( range( len( notes ) - 1 ) )
    
    d, note = notes[n]
    d_b, note_b = notes[n+1]
    
    d = min(d, d_b)/2
    if note>note_b:   
        new_notes = []
        while note - ( step := diatonic_lower_step_size(note) ) > note_b:
            note -= step
            new = (d, note)
            new_notes.append(new)
        notes = notes[:n+1] + new_notes + notes[n+1:]
        
    if note<note_b:
        new_notes = []
        while note + ( step := diatonic_upper_step_size(note) ) < note_b:
            note += step
            new = (d, note)
            new_notes.append(new)
        notes = notes[:n+1] + new_notes + notes[n+1:]
    genome = [prob_op, prob_mut, *notes]
    return genome    

In [235]:
passing_tone(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (1.5, 71),
 (1.5, 72),
 (1.5, 74),
 (1.5, 76),
 (1.5, 77),
 (1.5, 79),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [236]:
def delete_note(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    n = random.choice(range(len(notes)))
    
    notes = notes.copy()
    notes.pop(n)
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [237]:
delete_note(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (4, 77),
 (4, 71)]

In [238]:
def merge_note(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    is_consecutive = get_pairs(notes)
    
    if len(is_consecutive)==0: 
        return genome
    
    n = random.choice(is_consecutive)
    d_a, note_a = notes[n-1]
    d_b, note_b = notes[n]
    
    d = min(8, d_a + d_b)
    new = (d, note_a)
    
    notes[n-1] = new
    notes = notes.copy()
    
    notes.pop(n)
    genome = [prob_op, prob_mut, *notes]
    return genome

In [239]:
merge_note(gen)

[array([0.4, 0.2, 0.2, 0.2]),
 array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
        0.09090909]),
 (3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [272]:
def mutation(genome, *args):
    rules = [
        repeat,
        split,
        arpeggiate,
        leap,
        upper_neighbor,
        lower_neighbor,
        anticipation,
        delay,
        passing_tone,
        delete_note,
        merge_note
    ]
    
    prob_op, prob_mut, notes = split_genome(genome)
    mutation_rule = np.random.choice(
                        rules, size=1, p=prob_mut
                    )[0]    
    genome = mutation_rule(genome)
    return [genome]

In [270]:
mutation(gen)

[array([0.39045189, 0.14255212, 0.18181104, 0.28518495]),
 array([0.03640833, 0.00605717, 0.06913054, 0.2475582 , 0.07673454,
        0.09387541, 0.08775609, 0.08151578, 0.13514298, 0.08694019,
        0.07888077]),
 (3, 69),
 (5, 62),
 (3, 64),
 (4, 77),
 (4, 71)]

## Methods Regarding Crossover

In [280]:
def get_cutpoint(genome):
    prob_op, prob_mut, notes = split_genome(genome)
    duration = sum( note[0] for note in notes )
    
    point = random.choice(range(1, duration))
    return point

In [294]:
def cut_genome(genome, k):
    prob_op, prob_mut, notes = split_genome(genome)
    
    cum_time=0
    for i in range(len(notes)):
        d, note = notes[i]
        
        if cum_time == k:
            genome_left = [prob_op, prob_mut, *notes[:i]]
            genome_right = [prob_op, prob_mut, *notes[i:]]
            
            return genome_left, genome_right
        
        elif cum_time< k < cum_time+d:
            d_left = k - cum_time - 1
            d_right = d - d_left
            
            notes_left = notes[:i+1]
            notes_right = notes[i:]
            
            notes_left[-1] = (d_left, note)
            notes_right[0] = (d_right, note)
            
            genome_left = [ prob_op, prob_mut, *notes_left ]
            genome_right = [prob_op, prob_mut, *notes_right ]
            
            return genome_left, genome_right
        else:
            cum_time+=d

In [297]:
def merge_genomes( left, right ):
    prob_op_left, prob_mut_left, notes_left = split_genome(left)
    prob_op_right, prob_mut_right, notes_right = split_genome(right)
    
    prob_op = ( prob_op_left + prob_op_right ) / 2
    prob_mut = ( prob_mut_left + prob_mut_right) / 2
    notes = notes_left + notes_right
    
    genome = [prob_op, prob_mut, *notes]
    return genome

In [298]:
def crossover(genome, population_fitness, N ):
    k = max(2, N//5)
    other_genome = tournament(population_fitness, k)
    
    i = get_cutpoint(genome)
    j = get_cutpoint(other_genome)
    
    genome_left, genome_right = cut_genome(genome, i)
    other_left, other_right = cut_genome(other_genome, j)
    
    genome_a = merge_genomes(genome_left, other_right)
    genome_b = merge_genomes(other_left, genome_left)
    
    return [genome_a, genome_b]

In [295]:
l, r = cut_genome(gen, 15)
print( l[2:], r[2:] )

[(3, 69), (5, 81), (3, 64), (4, 77)] [(4, 71)]


In [287]:
gen[2:]

[(3, 69), (5, 81), (3, 64), (4, 77), (4, 71)]

## Methods for Duplication

In [393]:
def duplication(genome, *args):
    prob_op, prob_mut, notes = split_genome(genome)
    
    i = random.choice( range(len(notes)-1) )
    j = random.choice( range(1,8+1) )
        
    to_duplicate = notes[i : i+j]
    left = notes[ :i ]
    right = notes[ i+j: ]
    notes = left + to_duplicate + to_duplicate + right
    
    genome = [ prob_op, prob_mut, *notes]
    return [genome]

In [336]:
duplication(gen)[0][2:]

1 7
[(3, 69)] + [(5, 81), (3, 64), (4, 77), (4, 71)] + []


[(3, 69),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71),
 (5, 81),
 (3, 64),
 (4, 77),
 (4, 71)]

In [327]:
gen[2:]

[(3, 69), (5, 81), (3, 64), (4, 77), (4, 71)]

## Methods for Inversion

In [392]:
def inversion(genome, *args):
    prob_op, prob_mut, notes = split_genome(genome)
    
    i = random.choice( range(len(notes) - 1) )
    j = random.choice( range(i+1, len(notes) ) )
                      
    to_reverse = notes[i : j]
    left = notes[:i]
    right = notes[j:]
    
    notes = left + to_reverse[::-1] + right
    
    genome = [prob_op, prob_mut, *notes]
    return [genome]